# Homework 1: Reading data and regression
## Due September 5th

In class we've been working on developing the tools for linear regression.  In this homework, we'll use those tools to show that one of the original 'big data' problems can be solved quite credibly with ordinary least squares.  Note that this exercise is primarily intended to do two things: 1) provide you with some practice gathering data, and 2) evaluate your ability to reason somewhat abstractly about models.  With respect to the second point, it is in your grade's interest to provide complete and well-reasoned answers to narrative questions posed.  A correct answer can be awarded very few points if its reasoning is absent or unclear, and an incorrect answer can be awarded full points if it is well argued.   

Please turn in your work via github classroom.

## The big one
If you've ever lived in an area prone to seismic activity, you know that people are always a little apprehensive about *the big one*, that giant earthquake (Magnitude greater than 7) that is going to break the pots and collapse the overpasses.  However, the big one rarely happens... until it does.  What happens more frequently is smaller earthquakes.  And even more frequently than that, even smaller earthquakes.  In fact, it's long been understood that earthquake frequency has an inverse relationship with magnitude.  Here, we're going to quantify that relationship for the west coast of the US.  

## Data wrangling
The first thing that we'll need to do is to aquire a dataset that can help us say something about earthquake frequency.  Fortunately, the United States Geologic Survey keeps such a database.  

First, navigate to https://earthquake.usgs.gov/earthquakes/search/.  This is the USGS' central repository for earthquake data.  We'll be interested in data from the last twenty years.  Enter the appropriate date.  Next, we're interested in data from the west coast of the lower 48.  Use the Custom Geographic Region button on the right side of the page, followed by the Draw Rectangle on the Map button.  Draw a rectangle around the west coast, from the Canadian to Mexican border.  Next, open the Output Options tab and select .csv (comma separated values, a plain text format).  Finally, under Limit Results, enter 19999 (the site will throw an error if you make this value bigger).  This series of commands will deliver the ~20k most recent earthquakes of all sizes to occur in this region of the world.  It may take a moment for their server to pull your query together. Download the file.

Next, import the data into ipython.  This is easily done with [Pandas' read_csv function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

In [ ]:
# Use pandas to import the earthquake data file here
import pandas as pd
earthquakes = pd.read_csv('earthquakes.csv',sep=",",engine='python',index_col=0)

Interrogate the Pandas dataframe for the available fields using its built in method 'keys'


In [ ]:
earthquakes.keys()

We're primarily interested in determining the relationship between relative frequency and magnitude.  Extract the magnitude variable from the pandas data frame using the 'mag' key.  

Next, you'll need to determine the number of earthquakes that occured in the data as a function of magnitude.  This is easily done by deciding on a set of bins, and then counting the number of items in each bin.  This is also known as a [histogram](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html), and is easily computed using either numpy or matplotlib.  You'll want to use a fairly large number of bins, say 50 (equally spaced).  *Note that the histogram function provided by either of the above libraries returns total counts rather than relative frequencies: you'll need to compute relative frequencies by dividing the returned counts by the total number of earthquakes in the dataset.*

In [ ]:
mag = earthquakes['mag']
import numpy as np
hist, bins = np.histogram(mag, bins=50) # freq
relative_hist = hist / len(mag) # relative freq
# bin edges (len(hist) + 1)

**(20 pts) Make a plot with bin centroids on the $x$-axis and relative frequency on the $y$-axis.  Based on your results, please provide a few sentences describing whether the model $y = w_0 + w_1 x$ is a good fit to the data, if $x$ represents earthquake magnitude and $y$ represents relative frequency.**

!Answers go here

A linear model ($y = w_0 + w_1 x$) will not be a good fit to the data. Based on the plot, as the magnitude of an earthquake increases, the relative frequency seems to decrease exponentially. Therefore, a linear model would not be able to accurately depict the relationship between earthquake magnitude and relative frequency.

In [ ]:
import matplotlib.pyplot as plt
# get centroids
space = (bins[1] - bins[0]) / 2
centroid = np.array([x+space for x in bins][:len(bins)-1])

plt.plot(centroid,relative_hist,'ro')
plt.xlabel('Magnitude')
plt.ylabel('Relative Frequency')
plt.show()

As it turns out, a physical model that gives the predicted relative frequency of earthquakes of different magnitudes can be derived from physics, and is given by a so-called *power-law*
$$
y = ax^b
$$
**(20pts) If you wanted to directly fit this power-law model to these data, would you be able to use the linear regression code that we've already developed to do so?  If so, how?  If not, why not?**

!Answers go here

No you could not use the linear regression code to fit the power-law model to these data. Our current linear regression code is used to find the best fit for a model with a near-constant (linear) slope. The power-law model has a non-constant slope. Specifically for this data, the slope of the plot above (magnitude vs. relative frequency) starts off in a very steep negative direction and decreases in magnitude as earthquake magnitude increases. Our current linear regression code, which seeks to find the best constant slope for data, would not be able to capture this phenomena from a power-law model.

As it turns out, it is not possible to use linear regression to fit this model directly.  However, it can be done by using a clever transformation of the data.  **(20pts) Plot the base-10 logarithm of the event counts.  Now does a linear model seem like a good fit?  If the power law is a good fit to the untransformed data, why does it make sense that a linear model should fit the log-transformed data (HINT: take the logarithm of both sides of the power law)**

**(UPDATE!)** I should have noted this before: The earthquake magnitude $x$ has *already* been log-transformed, which is to say that a magnitude 7 is 10 times more powerful than a magnitude 6.  The power law stated above applies to the earthquake strength prior to it having the logarithm taken.  As such, a more reasonable way to state the question would be as follows: A physical model that gives the predicted relative frequency of earthquakes of different magnitudes can be derived from physics and is given by a so-called *power-law*
$$
y = a P^b,
$$
where $P=10^x$. Show that taking the logarithm of both sides of this equation leads to a linear relationship between the earthquake magnitude $x$ and the log-frequency $\text{log}_{10}(y)$. 

!Answers go here

We start with $y = a P^b = a 10^{bx}$.

Taking the logarithm of both sides we get $\text{log}_{10}(y) = \text{log}_{10}(a 10^{bx})$.

Then, using the logarithm product rule we get $\text{log}_{10}(y) = \text{log}_{10}(a) + \text{log}_{10}(10^{bx})$.

So, $\text{log}_{10}(y) = \text{log}_{10}(a) + bx$.

Thus, the log-frequency is linearly related to the earthquake magnitude with a slope of $b$ and y-intercept of $\text{log}_{10}(a)$.

In [ ]:
# plot base-10 log of data
# use event counts
hist_transformed = np.log10(hist+1) # for values with 0
plt.plot(centroid,hist_transformed,'ro')
plt.xlabel('Magnitude')
plt.ylabel('Relative Frequency')
plt.show()
print(hist)
print(hist_transformed)

Finally, **(40pts) use one of the methods that you developed for linear regression in class to fit a line to the log-transformed counts.  What is the slope of that line?  Is the model a good fit for all sizes of earthquake?**

!Answers go here

In [ ]:
def gradient(w):
    resid = hist_transformed - w[1]*centroid - w[0]
    dI_dw0 = -np.sum(resid)
    dI_dw1 = -np.sum(resid*centroid)
    return np.array([dI_dw0, dI_dw1])

In [ ]:
w = np.array([5, -1]) # w0, w1
s = 0.0001 # step size
tol = 0.001 # grad_I size tolerance
grad_I = gradient(w) # initial gradient

In [ ]:
print(grad_I)
while np.linalg.norm(grad_I) > tol:
    w = w - s*grad_I
    grad_I = gradient(w)
print("w:", w)